# A brief description of the proposed distribution

They calculate different characteristics of the voxel values of images captured by different devices, and using statistical hypothesis testing they reject null hypothesizes by low values of p-values, and they conclude that the statistical distribution generating these data should have below features:

1. **Positive skewness for low attenuation levels that gradually decreases.**
2. **Leptokurtic (more concentrated around the mean than the normal distribution) behavior for low attenuation levels that gradually normalizes for higher attenuation levels.**
3. **Linear relationship between mean and variance.**
4. **Linear relationship between kurtosis and skewness^2 (with convergence to the Gaussian distribution, related to the first one).**

So, they propose the non-central gamma distribution - with three variables of alpha, beta, and delta - in which:
1. **skewness = 2/sqrt(alpha)**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; For higher attenuation levels they propose bigger alphas, so skewness gradually decreases.

2. **The excess of kurtosis is positive and gradually decreases (it converges to a Gaussian distribution). Excess Kurtosis of gamma = 6/alpha**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; As above, alpha goes bigger and bigger due to attenuation level's raising.

3. **variance = beta * (mean - delta)**


4. **kurtosis = (3 * skewness ^ 2) / 2 + 3**

# Global-Local approach
First, they use the Expectation-Maximization method with fixed initial values of theta on all voxels to have a good initial theta before calculating the true theta for each neighborhood. Another benefit of this approach is that major components in the whole image become more important than the other small components. (eq13 and eq14 of the paper)

![](../resources/figs/eq.13.png)

![](../resources/figs/eq.14.png)



# Expectation-Maximization
This approach is often used to cluster the mixture-model of the number of distributions from one kind. As I searched, it is more used with Gaussian mixture-model.

They define a set of random variables, Z, that if its member like Z_i is equal to j, determines Y_i belongs to the jth component.

In the expectation step, we want to find corresponding distribution for each voxel value (the responsibilities based on the current parameters).
As you can see below: (eq18 of the paper)

(y_i is the ith voxel value)

![](../resources/figs/eq.18.png)


But, I think it would be more comprehensible if we write eq.18 this way: 

![](../resources/figs/eq.18.r.png)


Then, in the maximization step, we know that each voxel probably is from which distribution, then we try to find new parameters.

Now, we should maximize the expectation of the data likelihood: (eq16 of the paper)

![](../resources/figs/eq.16.png)

Where likelihood is:

![](../resources/figs/likelihood.png)


So, if we want the expected value for likelihood (or log-likelihood), we should calculate the expected value for all z values (all j components): (eq17 of the paper)

![](../resources/figs/eq.17.png)


And because both of the result terms are positive, the maximization step can be performed independently.

After optimization, they got: (eq22 of the paper)

![](../resources/figs/eq.22.png)

(eq24 of the paper)

![](../resources/figs/eq.24.png)


By solving the equation above, we can have α, and then with the constraint that αβ = μ, we can have β.


# Local non-central gamma
The calculations for the local non-central gamma is very similar to the global non-central gamma, with the difference that this model works on a neighborhood of a particular voxel and uses the global theta as its initial values for calculating its true theta.

### Some estimations
They use sample conditioned local moments as estimations for conditioned moments. Like the below image, they simply sum over all components:

![](../resources/figs/estimations.png)



# Variance stabilization
In the next step of their paper, they want to stabilize the variance of voxels. For this goal, they want to use a transform by which the variance of voxels values after the transformation becomes independent from its mean value. (As the point mentioned above, for non-central gamma distribution we have variance = beta * (mean - delta))

So, they define a variable mapping using a differentiable function f. 
`Y = f(X)`

X_n is a sequence of random variables that satisfies:

`sqrt(n) * (X_n - μ) -d-> Ν(0, σ^2)`

* -d->: convergence in distribution
According to the "All of statistics" book (wasserman):
![](../resources/figs/convergence_in_dist.png)
F is CDF for X and F_n is CDF for X_n

Because f is differentiable, for the first order Taylor expansion of f around μ we can write:

`f(X_n) = f(μ) + f'(μ) * (X_n - μ)` => `f(X_n) - f(μ) = f'(μ) * (X_n - μ)` => `sqrt(n) * (f(X_n) - f(μ)) = f'(μ) * sqrt(n) * (X_n - μ)`

for the right-hand-side we have:

`f'(μ) * sqrt(n) * (X_n - μ) -d-> Ν(0, σ^2 * f'(μ)^2)`

So, we conclude that:

`sqrt(n) * (f(X_n) - f(μ)) -d-> Ν(0, σ^2 * f'(μ)^2)`

Now, suppose that the variance (σ^2) is a function of the mean (μ). Some function that the authors prefer to represent it with `Var`. So, we have `σ^2 = Var(μ)`.

So, from the all possible functions for f, we want a subset of them in which the value `σ^2 * f'(μ)^2` is a constant. We represent them with f_stab. So:

`σ^2 * f_stab'(μ)^2 = C` => `f_stab'(μ) = sqrt(C) / σ`

using integral we have:

`f_stab(y) = C * sqrt(y/β) + K`
* K and C are arbitrary constants.

So, we got this:
(eq35 of the paper)

![](../resources/figs/eq.35.png)


The transform of sqrt on the variable Y leads to a generalized Gamma distribution suggested by Stacy (1962) that its pdf is known. Also, its moments calculation follows a certain formula (suggested by authors in their previous works):

`W = sqrt(Y)`

![](../resources/figs/eq.37.png)

From another paper written by Tricomi and Erdelyi (1951), we have the variance of this transform: (eq38 of the paper)

`W = C * sqrt(Y/β) + K` (constant K does not affect the variance)

![](../resources/figs/eq.38.png)

<hr>
I took a look at Tricomi and Erdelyi's paper, their goal is to find the asymptotic behavior of below when the z grows so large:

![](../resources/figs/Tricomy.png)

And, I found that the last term of the right-hand-side in eq38 is big O which is often used to indicate the order. It means that the function is less than or equal to something related to 1/α^2, So in big values of α, it becomes negligible.
<hr>

So, for a variance of 1, we should set C = 2.

But, eq35 is not enough. Because we want a transform in which variance should be stabilized (this one is met) AND averages should not change. For this purpose, they suggest the below equation as a good transform: (eq40 of the paper)

![](../resources/figs/eq.40.png)

They do not provide any description for this equation, but I think it is quite easy if we think of using the central limit theorem. Look at the below plots: (They are not precise, but approximately they are acceptable)

![](../resources/figs/before_anything.png)

![](../resources/figs/after_sqrt.png)


![](../resources/figs/after_shift.png)


![](../resources/figs/after_variance_change.png)



![](../resources/figs/after_shift_back.png)



<br>

<br>

<br>

<br>


Previously we said that <.|Z(r)=j> is an estimation for E{.|Z(r)=j}, So:

(eq41 of the paper)

![](../resources/figs/eq.41.png)

Now, as what we use in local moments before, we estimate the moments in a similar way:

(eq42 of the paper)

![](../resources/figs/eq.42.png)


The paper does not include this part for calculation of Var{sqrt(Y)}, but it is pretty easy, because:


![](../resources/figs/var_calc.png)


So, from the expected value estimations we can have the estimation of variance.